# Example 3: Maximization of a multivariate function 

In this notebook the framework is used to optimize a function which depends on two variables $x_0$ and $x_1$. We illustrate both closed-loop and iterative approaches to solving the optimization problem using our framework. 

Similar to Example 2, we use here a known function to illustrate how the framework is applied, but it is not a requirement for the framework that the function can be written down explicitly as we do below. Hence, this option of using the framework allows for optimizing _unknown_ functions or _functions without explicit definitions_ and will come in handy for e.g. optimizing complex experiments. 

In the following we will maximize the Easom function, a classical optimization test function
$$
f(\mathbf{x}) = - \cos{(x_0)} \, \cos{(x_1)} \, \mathrm{e}^{-(x_0 - x_0^*)^2 - (x_1 - x_1^*)^2}, \quad \mathbf{x} = [x_0, x_1].
$$

It is known that the function above has its maximum at $\mathbf{x}^* = (x_0^*, x_1^*)$ (typical values used for the minimum are $\mathbf{x}^* = (\pi, \pi)$ but we will use $\mathbf{x}^* = (0,0)$). For more on the Easom function, see this page [https://www.sfu.ca/~ssurjano/easom.html](https://www.sfu.ca/~ssurjano/easom.html).


## Framework approach

We will solve the problem in two different ways:
1. using the closed-loop approach of the `.auto`-method
2. using the iterative optimization approach of the framework which requires using the methods `.ask` and `.tell`. This approach allows for iterative optimization and optimization of any callable function, known or otherwise.

The optimization process can be stopped after any number of iterations.


## Technical note

Installation of `torch` and `torchvision` (required dependencies) cannot be bundled as part of the `creative_project` installable. This is unfortunate, but a known issue it seems. Therefore these must be installed first, before installing `creative_project`.

### Get started: Import libraries

In [ ]:
# Preamble

# Install torch and torchvision. Use this link to identify the right versions to install on your system 
# depending on configuration: https://pytorch.org/get-started/locally/

pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

# Install creative_project from github (will require authentication with password)
pip install --user https://github.com/svedel/kre8_core/

In [ ]:
# import
import numpy as np
import torch
from creative_project import CreativeProject
import matplotlib.pyplot as plt
%matplotlib inline

### Setup the problem to be solved: define response function

Define the problem to optimize. Here we define a known function and use both the closed-loop solution method based on the `.auto`-method which requires the function to be specified as well as the iterative approach using the `.ask` and `.tell` methods that can be applied to both specified and unspecified, but sample-able functions that cannot be written down.

In [ ]:
# define the function (negative of the Easom function)
def neg_Easom(x):
    return torch.cos(x[0]) * torch.cos(x[1]) * torch.exp(-(x[0] ** 2 + x[1] ** 2))

x = torch.linspace(0,1)
plt.figure(figsize=(8, 4))
plt.plot(x.numpy(), neg_Easom(x).numpy())
plt.show()

Set up the problem

In [ ]:
# define the range of interest
x0_init = 1
x1_init = -1
covars2d = [(x0_init, -5, 5), (x1_init, -5, 5)]

### Solution 1: Closed-loop solution approach using `.auto` method

Instantiate the `CreativeProject` class and solve the problem

In [ ]:
# initialize class instance
cc = CreativeProject(covars=covars2d)

# number of iterations
max_iter = 20

# run the auto-method
cc.auto(response_samp_func=neg_Easom, max_iter=max_iter)

Best guess after solving

In [ ]:
# run current_best method
cc.current_best()

### Solution 2: Iterative solution using `.ask` and `.tell` methods

Instantiate the `CreativeProject` class and solve the problem. In this case, we need to write our own loop to iterate. Notice that the `covars` and `response` variables are converted to `torch` tensors of size $1 \times \mathrm{\#covariates}$ to store them in the instantiated class, where they are used for retraining the model at each iteration.

In [ ]:
# initialize the class instance
cc2 = CreativeProject(covars=covars2d)

# run the solution
for i in range(max_iter):
    # generate candidate
    cc2.ask()

    # sample response
    covars = torch.tensor([[it.item() for it in cc2.proposed_X[-1]]], dtype=torch.double)
    response = torch.tensor([[neg_Easom(cc2.proposed_X[-1]).item()]], dtype=torch.double)

    # report response
    cc2.tell(covars=covars, response=response)

Best guess after solving

In [ ]:
# run current_best method
cc2.current_best()